In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os



In [197]:
from strategy_engine.trade_constructor import Trade, StoreTrades
from strategy_engine.trade_constructor import trades
from strategy_engine.trade_constructor import market_data
from strategy_engine.trade_constructor import trade_data

In [198]:
trade_data.head()

,QUOTE_DATE,EXPIRE_DATE,STRIKE,action,position_type,confidence,entry_price
0,2023-08-01,2023-08-01,370.0,SELL,SHORT_PUT,0.999911,0.005
1,2023-08-01,2023-08-01,380.0,SELL,SHORT_PUT,0.999911,0.005
2,2023-08-01,2023-08-01,388.0,SELL,SHORT_PUT,0.999912,0.005
3,2023-08-01,2023-08-01,390.0,SELL,SHORT_PUT,0.999877,0.005
4,2023-08-01,2023-08-01,395.0,SELL,SHORT_PUT,0.999775,0.005


In [199]:
trade_data.columns

Index(['QUOTE_DATE', 'EXPIRE_DATE', 'STRIKE', 'action', 'position_type',
       'confidence', 'entry_price'],
      dtype='object')

In [200]:
market_data.columns

Index(['QUOTE_DATE', 'QUOTE_TIME_HOURS', 'UNDERLYING_LAST', 'EXPIRE_DATE',
       'DTE', 'STRIKE', 'C_BID', 'C_ASK', 'C_IV', 'C_DELTA', 'C_GAMMA',
       'C_THETA', 'C_VEGA', 'C_RHO', 'C_VOLUME', 'P_BID', 'P_ASK', 'P_IV',
       'P_DELTA', 'P_GAMMA', 'P_THETA', 'P_VEGA', 'P_RHO', 'P_VOLUME',
       'STRIKE_DISTANCE', 'STRIKE_DISTANCE_PCT'],
      dtype='object')

In [ ]:
class BacktestEngine:
    def __init__(self, trades, trade_data: pd.DataFrame, market_data: pd.DataFrame):
        self.trades = trades
        self.trade_data = trade_data
        self.market_data = market_data
        
    def run(self):
        
        results = []
        
        for trade in self.trades:
            try:
                entry_date = self.trade_data['QUOTE_DATE'] == trade.entry_date
                exit_date = self.trade_data['EXPIRE_DATE'] == trade.exit_date
                strike = self.trade_data['STRIKE'] == trade.strike
                entry_price = self.trade_data['entry_price'] == trade.entry_price
                exit_price = self.market_data[['UNDERLYING_LAST']][self.market_data['QUOTE_DATE']==trade.entry_date] && ['EXPIRE_DATE'==trade.exit_date] && ['STRIKE'==trade.strike]]
                
                    
